In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount('/content/drive')
import os
drive_path='/content/drive/Mydrive/'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:


# Define paths
base_dir = '/content/drive/MyDrive/dataset'
csv_path = os.path.join(base_dir, 'test.csv')  # Adjust if your CSV file is named differently
image_dir = base_dir  # Images are inside the subfolders of `dataset`

# Load CSV file
df = pd.read_csv(csv_path)

# Map image paths to full paths
df['image_path'] = df['Image_id'].apply(lambda x: os.path.join(image_dir, x))
df = df[df['image_path'].apply(lambda x: os.path.exists(x))]  # Filter rows where images exist

# Check data
print(f"Number of images in dataset: {len(df)}")
print(df.head())

# Split the dataset into training and validation sets
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['labels'], random_state=42)

# Define parameters for image processing
img_height, img_width = 224, 224
batch_size = 32

# Use ImageDataGenerator to load and augment images
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1.0/255)

# Create training and validation generators
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='image_path',
    y_col='labels',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_dataframe(
    val_df,
    x_col='image_path',
    y_col='labels',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Verify the class indices
print("Class Indices:", train_generator.class_indices)

Number of images in dataset: 0
Empty DataFrame
Columns: [Image_id, labels, image_path]
Index: []


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [3]:


# Define the dataset path
base_dir = '/content/drive/MyDrive/dataset'  # Replace with the actual dataset folder path

# Define parameters for image processing
img_height, img_width = 224, 224  # Image size for input
batch_size = 32

# Use ImageDataGenerator to load and augment images
train_datagen = ImageDataGenerator(
    rescale=1.0/255,          # Normalize pixel values to [0, 1]
    rotation_range=30,        # Randomly rotate images
    width_shift_range=0.2,    # Shift images horizontally
    height_shift_range=0.2,   # Shift images vertically
    shear_range=0.2,          # Apply shear transformation
    zoom_range=0.2,           # Apply zoom
    horizontal_flip=True,     # Flip images horizontally
    validation_split=0.2      # Use 20% of the data for validation
)

# Create training and validation generators
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Use training subset
)

val_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Use validation subset
)

# Verify class indices
print("Class Indices:", train_generator.class_indices)

Found 1224 images belonging to 6 classes.
Found 306 images belonging to 6 classes.
Class Indices: {'alien_test': 0, 'cloudy': 1, 'foggy': 2, 'rainy': 3, 'shine': 4, 'sunrise': 5}


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

# Data Augmentation and Generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'path_to_train_dataset',     # Replace with your training dataset path
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    'path_to_val_dataset',       # Replace with your validation dataset path
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Model Definition
model = Sequential([
    Input(shape=(224, 224, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')  # 6 classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Calculate steps per epoch
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = val_generator.samples // val_generator.batch_size

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[early_stopping]
)

# Save the model
model.save('weather_classification_model.h5')

FileNotFoundError: [Errno 2] No such file or directory: 'path_to_train_dataset'

In [ ]:
# Save the model
model.save('weather_classification_model.keras')

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from collections import Counter

# Define the dataset path
base_dir = '/content/drive/MyDrive/dataset'  # Replace with your dataset folder path

# Define parameters for image processing
img_height, img_width = 224, 224  # Image size for input
batch_size = 32

# Data Augmentation and Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255,          # Normalize pixel values to [0, 1]
    rotation_range=30,        # Randomly rotate images
    width_shift_range=0.2,    # Shift images horizontally
    height_shift_range=0.2,   # Shift images vertically
    shear_range=0.2,          # Apply shear transformation
    zoom_range=0.2,           # Apply zoom
    horizontal_flip=True,     # Flip images horizontally
    validation_split=0.2      # Use 20% of the data for validation
)

# Training and Validation Generators
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Use training subset
)

val_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Use validation subset
)

# Check class indices
print("Class Indices:", train_generator.class_indices)

# Define the CNN Model
model = Sequential([
    Input(shape=(img_height, img_width, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')  # Adjust output layer for the number of classes
])

# Compile the Model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Define Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_weather_model.keras', save_best_only=True, monitor='val_loss', mode='min')

# Train the Model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=val_generator.samples // val_generator.batch_size,
    callbacks=[early_stopping, model_checkpoint]
)

# Save the final model
model.save('final_weather_classification_model.keras')

# Evaluate the Model
loss, accuracy = model.evaluate(val_generator)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

# Make Predictions
import numpy as np

# Predict on validation set (example)
sample_images, sample_labels = next(val_generator)
predictions = model.predict(sample_images)

# Decode Predictions
predicted_classes = np.argmax(predictions, axis=1)
actual_classes = np.argmax(sample_labels, axis=1)

print(f"Predicted Classes: {predicted_classes}")
print(f"Actual Classes: {actual_classes}")

Found 1224 images belonging to 6 classes.
Found 306 images belonging to 6 classes.
Class Indices: {'alien_test': 0, 'cloudy': 1, 'foggy': 2, 'rainy': 3, 'shine': 4, 'sunrise': 5}
Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 221s 6s/step - accuracy: 0.3913 - loss: 1.6969 - val_accuracy: 0.6076 - val_loss: 1.3149
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - accuracy: 0.6250 - loss: 1.4739 - val_accuracy: 0.6111 - val_loss: 1.2011
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 216s 5s/step - accuracy: 0.5707 - loss: 1.1263 - val_accuracy: 0.6146 - val_loss: 1.1347
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 40ms/step - accuracy: 0.5312 - loss: 1.1415 - val_accuracy: 0.5556 - val_loss: 1.0136
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 269s 6s/step - accuracy: 0.6457 - loss: 0.9185 - val_accuracy: 0.6806 - val_loss: 0.8856
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.7188 - loss: 0.9111 - val_accuracy: 0.5556 - val_loss: 1.0751
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 241s 5s/step - accura